# IMA 205 - TP ANN (part C) - OPTIONAL

## Coding a Multi-Layer Perceptron with Keras/Tensorflow

This part is optional, and you do not have to put your solution on eCampus. Regardless, I encourage you to do this part if you have spare time during the lab as it gives you a chance to familiarize yourself with the higher-level API of Keras. You will see that when the task is straightforward and involves only pre-existing building blocks, implementing it via Keras is very fast.

Original Author : Alasdair Newson

Adapted by Loic Le Folgoc. If you have questions, you can contact me at loic.lefolgoc@telecom-paris.fr

In the second part of this TP, we will be looking at the same Multi-Layer Perceptron (MLP) practical work, using Keras (https://keras.io/)

We will be using the following packages :
   
   - Scikit-learn (http://scikit-learn.org/)
   - Keras (https://keras.io/)

The following commands will make sure that you have all the necessary packages

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import os
import pdb

import sklearn  # scikit-learn
import keras

from keras import layers
print(keras.__version__)

In [ ]:
# This is the backend used by Keras by default:
keras.backend.backend()

# 1 - Multi-Layer Perceptron with Keras

## Introduction to Keras

Keras is a high-level, deep learning API originally written by François Chollet and now developed by Google for implementing neural networks. Like Pytorch, it provides automatic differentiation so that the user does not have to determine gradients manually, which can be extremely long even for simple networks, as you have seen in the previous lab.

Previously, Keras was a high-level API using Tensorflow as a backend. Tensorflow is a library written by Google which also allows implementation of deep neural networks. Previously, Tensorflow required a strict separation between the creation (declaration) of variables and their execution (giving a numerical value, using the ```Session``` function). With Keras and Tensorflow 2, this separation has been removed, and the language has become simpler, in particular for creating and training networks.

Since Keras 3, Keras can now be used on top of any backend in a transparent manner: JAX, Pytorch, or Tensorflow. By default, it still uses Tensorflow (as in our code below): but you will notice the code itself makes no explicit reference to Tensorflow.


## Creating a network

To create a network, there are two (main) methods :

### Creating a model using the Sequential API

Keras has a simple way of adding layers to create a neural network. First, you can indicate to Keras that the model is 'sequential' (a simple model, with not many tweaks). For this, you can use the following function :
- ```model = keras.Sequential()```

After this, you can add layers with the function.

- ```model.add()```

You can then use the ```layers.Dense``` and ```Activation``` functions to specify different layer types.  __Note that in the case of this approach, you will have to specify the input image size in the first layer of the network, inside the first layer function.__ So, for example, if the first layer is a dense layer with a relu activation, with n_out output neurons, and n_input neurons :

- `model = keras.Sequential()`
- `model.add(layers.Dense(n_out,activation,input_shape=(n_in,),activation='relu')`

Otherwise, the network does not know how many weights to create. __Be careful of this special case (the first layer)__.

Notice that here, the activation is simply an argument of the `Dense` layer.<br>

Alternatively, like in Pytorch, we can pass the layers as arguments to `Sequential`: `model = keras.Sequential([layers.Dense(...),layers.Dense(...)])`.

### Creating a model using the standard API

Otherwise, another approach to creating the model is to explicitly create the input variable, and just cascade the different functions, as in Tensorflow. So, for the same example, we would have :

- `input = keras.Input(shape=(n_in,))`
- `output = layers.Dense(n_in,n_out,activation='relu')(input)`
- `model = keras.Model(input, output)`


For now, let's use the Sequential API (however, if you want to try the standard API, go ahead).

# MLP with Keras : Architecture

We are now going to create an MLP with Keras. We will start with an MLP with one hidden layer. The network should consist of the following operations, in the following order :

- Fully connected layer, with 50 output neurons
- ReLU activation
- Fully connected layer, with 1 output neuron (because we have a binary classification problem)
- Sigmoid output activation

We are going to be solving a binary classification problem, so the output of the network should be a scalar between 0 and 1 (thus the last Sigmoid activation function).

The loss function should be defined as the binary cross-entropy between the predicted class and the true class

# Training and testing a network

Tensorflow allows the easy training of a network with the following functions :

- `model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=learning_rate))` : create the loss function and define the optimizer used. This function can do many other useful things (such as specifying different metrics to look at the model's performance e.g., with the argument: `metrics=["accuracy"]`)
- `model.fit(...)` : train the model
- `model.evaluate(...)` : test the model
- `model.predict(...)` : carry out a simple forward pass on the model

# Load data

First, we load the data.

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X, Y = datasets.make_circles(n_samples=1000, noise=0.05, factor=0.5)
X = MinMaxScaler().fit_transform(X)

#show data in plot
plt.plot(X[Y == 1, 0], X[Y == 1, 1], 'ro')
plt.plot(X[Y == 0, 0], X[Y == 0, 1], 'bo')
plt.grid('on')

We are now going to define some useful auxillary functions.

First, a function that shows the decision boundary of our network. This works only for 2D input data

In [ ]:
# THIS CODE IS GIVEN

def plot_decision_function_2d(model_mlp, X, Y):
   # create a mesh to plot in
    h = .02  # step size in the mesh
    offset = 0.1
    x_min, x_max = X[:, 0].min() - offset, X[:, 0].max() + offset
    y_min, y_max = X[:, 1].min() - offset, X[:, 1].max() + offset
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    Z = model_mlp.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z<0.5
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired)

    plt.plot(X[Y == 1, 0], X[Y == 1, 1], 'yo')
    plt.plot(X[Y == 0, 0], X[Y == 0, 1], 'ko')

    plt.title("Decision surface")
    plt.axis('tight')

## CREATING AND TRAINING THE MODEL

We are now ready to create our network with Keras and to carry out training on our training dataset.

In [ ]:
# FILL IN CODE BY STUDENTS IN THIS SECTION

# We split up the data into training and test data, using a function from Scikit-learn :
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
print(X_train.shape)

# Layer sizes
n_input = X.shape[1]  # Number of input features
n_hidden = 50  # Number of neurons in the hidden layer
n_output = 1 # Number of output neurons

# create the model (using the sequential API)

model_mlp = keras.Sequential([...])   # FILL IN STUDENTS

# create the loss and optimiser
learning_rate = 0.01
model_mlp.compile(loss=..., optimizer=...,metrics=["accuracy"]) # TO FILL IN

# Run optimisation algorithm
n_epochs = 30
batch_size = 64

print('Training')
model_mlp.fit(..., ..., epochs=...,batch_size=...) # TO FILL IN

print('Testing')
model_mlp.evaluate(...,  ..., verbose=2) # TO FILL IN


In [ ]:
plot_decision_function_2d(model_mlp, X_test, Y_test)

# 2 - MLP with several hidden layers, on MNIST

We are now going to create an MLP with several hidden layers. We are going to use a more complicated dataset : the MNIST dataset, which contains images of handwritten digits. There are 10 classes, one for each digit.

We are going to implement the following architecture :

- Fully connected layer
- Relu activation
- Fully connected layer
- Relu activation
- Fully connected layer
- Relu activation
- Fully connected layer
- Softmax output activation

__IMPORTANT Note__ The sigmoid layer has been replaced by a softmax layer (at the end). This is normal, since we have a multi-class problem. __However, this differs from our Pytorch implementation, because Pytorch includes the softmax in the cross-entropy loss, unlike Keras.__


First, we load the MNIST dataset.

In [ ]:
# Import MNIST data
mnist = keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

Y_train = keras.utils.to_categorical(y_train)  # in order to convert y to a matrix with (num_examples, num_classes) (one-hot encoding)
Y_test = keras.utils.to_categorical(y_test)  # in order to convert y to a matrix with (num_examples, num_classes) (one-hot encoding)

#reshape the input images : flatten the last two dimensions
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1]*X_test.shape[2]))


Finally, fill in the following code to create and train your MLP

In [ ]:

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_hidden_3 = 128 # 3rd layer number of neurons

n_input = X_train.shape[1]
n_classes = Y_train.shape[1] # MNIST total classes (0-9 digits)

# TO CODE BY STUDENTS


model_mlp_multi_layer = ...

# create the loss and optimiser, use keras.losses.categorical_crossentropy in loss
...

# Run optimisation algorithm
n_epochs = 20
batch_size = 64

print('Training')
...

print('Testing')
...

In [ ]:
# THIS CODE IS GIVEN

def test_mnist_images(model_mlp_multi_layer,X_test):
  plt.figure(figsize=(10, 6))
  for idx in range(0,10):
      plt.subplot(2, 5, idx+1)
      rand_ind = np.random.randint(0,X_test.shape[0])
      plt.imshow(np.reshape(X_test[rand_ind,:],(28,28)),cmap='gray')
      # get prediction
      model_prediction = np.argmax(model_mlp_multi_layer.predict(np.expand_dims( X_test[rand_ind,:], axis=0)),axis=1)
      plt.title(int(model_prediction))

In [ ]:
test_mnist_images(model_mlp_multi_layer,X_test)

You may observe some difficulties in getting good performance in this case. For me, this was much more obvious here than in the Pytorch implementation for some reason (maybe different implementation choices: in the initialization of the layers, the optimizer, etc.). To improve training, we can turn to __regularisation__.

## BATCH NORMALISATION

One approach to improving the convergence of neural network training which we have seen during the lesson is known as batch normalisation, which we have seen during the lesson. This can be implemented very simply in Tensorflow by adding the following layer :

- ```layers.BatchNormalization()```

Change your model below, and implement this using your code above

In [ ]:
# TO CODE BY STUDENTS

...

In [ ]:
test_mnist_images(model_mlp_multi_layer,X_test)

__Question__: do you observe improved convergence ?

## Dropout

Often in the process of training neural networks, there is a difference between the training accuracy and the testing accuracy. This is the problem of overfitting. To alleviate this problem, we can turn to dropout:

`layers.Dropout(rate,...)`

where rate is the probability that an input neuron to a layer will get set to 0. You can do this in any layer. Use the previous architecture and try out the dropout. Set the dropout rate to 0.1 on the first layer as a test. Then try it out on all layers (you may not see a great difference here since the mnist database is relatively simple, this is just to show you how it is done)

In [ ]:
dropout_rate = 0.1

# TO CODE BY STUDENTS

...


#### Documentation:

  - https://keras.io/
  - http://www.deeplearningbook.org/